In [1]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using MacroEnergy
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
#using JLD2, FileIO
using JuMP

  Activating project at `~/Macro`

libgomp: Invalid value for environment variable OMP_NUM_THREADS: invalid

libgomp: Invalid value for environment variable OMP_NUM_THREADS: invalid


In [2]:
# Function to get sorting key: returns the index of the first matching suffix or a large number
function suffix_sort_key(name)
    idx = findfirst(s -> endswith(name, s), suffixes)
    return isnothing(idx) ? length(suffixes) + 1 : idx  # Default to placing non-matching names at the end
end

# Function to get flows for each type of asset
function get_flows(asset::MacroEnergy.CementPlant, df::DataFrame)
    df[:, "trad_" * string(asset.id) * "_cement"] = MacroEnergy.value.(MacroEnergy.flow(asset.cement_edge)).data
    df[:, "trad_" * string(asset.id) * "_co2"] = MacroEnergy.value.(MacroEnergy.flow(asset.co2_edge)).data
end

function get_flows(asset::MacroEnergy.ElectrochemCementPlant, df::DataFrame)
    df[:, "echem_" * string(asset.id) * "_cement"] = MacroEnergy.value.(MacroEnergy.flow(asset.cement_edge)).data
end

function get_flows(asset::ElectricDAC, df::DataFrame)
    df[:, string(asset.id) * "_co2"] = MacroEnergy.value.(MacroEnergy.flow(asset.co2_edge)).data
end

function get_flows(asset::PowerLine, df::DataFrame)
    df[:, asset.id] = MacroEnergy.value.(MacroEnergy.flow(asset.elec_edge)).data
end

function get_flows(asset::Battery, df::DataFrame)
    df[:, string(asset.id) * "_charge"] = -1 * MacroEnergy.value.(MacroEnergy.flow(asset.charge_edge)).data
    df[:, string(asset.id) * "_discharge"] = MacroEnergy.value.(MacroEnergy.flow(asset.discharge_edge)).data
end

function get_flows(asset::ThermalPower, df::DataFrame)
    df[:, asset.id] = MacroEnergy.value.(MacroEnergy.flow(asset.elec_edge)).data
    df[:, string(asset.id) * "_co2"] = MacroEnergy.value.(MacroEnergy.flow(asset.co2_edge)).data
end

function get_flows(asset::VRE, df::DataFrame)
    df[:, asset.id] = MacroEnergy.value.(MacroEnergy.flow(asset.edge)).data
end

get_flows (generic function with 7 methods)

In [3]:
case_path = @__DIR__
lazy_load = false

false

In [12]:
# Run single case
(system, model) = run_case(@__DIR__; optimizer=Gurobi.Optimizer);

###### ###### ######
Running case at /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_retrofit


┌ Info: Loading system from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_retrofit/system_data.json
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/generate_system.jl:18
┌ Warning: WeightTotal not found in time_data.json
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_time_data.jl:81
┌ Info: Using PeriodLength as default value for WeightTotal
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_time_data.jl:82
┌ Info: Done generating system. It took 0.64 seconds
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/generate_system.jl:35
┌ Info: Done loading system. It took 0.64 seconds
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/lo

Set parameter LicenseID to value 197246
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "license.rc.princeton.edu"
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Red Hat Enterprise Linux 8.10 (Ootpa)")

CPU model: Intel(R) Xeon(R) Gold 6246R CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 867266 rows, 604486 columns and 2898516 nonzeros
Model fingerprint: 0xcf933e86
Coefficient statistics:
  Matrix range     [1e-03, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 421042 rows and 219564 columns
Pre

┌ Info: Scaling constraints and RHS
└ @ MacroEnergy /home/al3792/Macro/src/utilities/run_tools.jl:26
┌ Info: Writing capacity results to /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_retrofit/results/capacity.csv
└ @ MacroEnergy /home/al3792/Macro/src/write_outputs/capacity.jl:118
┌ Info: Writing costs to /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_retrofit/results/costs.csv
└ @ MacroEnergy /home/al3792/Macro/src/write_outputs/costs.jl:45


In [13]:
capacity_results_df = MacroEnergy.get_optimal_capacity(system)

Row,commodity,commodity_subtype,zone,resource_id,component_id,type,variable,value
,Symbol,Symbol,Symbol,Symbol,Symbol,Symbol,Symbol,Float64
1,Cement,capacity,cement_produced,cement_MA,cement_MA_cement_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},capacity,87.2996
2,Cement,capacity,cement_produced,cement_CT,cement_CT_cement_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},capacity,87.497
3,Cement,capacity,cement_produced,cement_ME,cement_ME_cement_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},capacity,86.775
4,Cement,capacity,cement_produced,cement_MA,cement_MA_cement_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},capacity,26.4325
5,Cement,capacity,cement_produced,cement_CT,cement_CT_cement_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},capacity,26.5228
6,Cement,capacity,cement_produced,cement_ME,cement_ME_cement_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},capacity,25.9526
7,Electricity,capacity,elec_MA_elec_CT,MA_to_CT,MA_to_CT_elec_edge,PowerLine,capacity,2955.33
8,Electricity,capacity,elec_MA_elec_ME,MA_to_ME,MA_to_ME_elec_edge,PowerLine,capacity,2000.0
9,Electricity,capacity,elec_MA,storage_MA,storage_MA_discharge_edge,Battery,capacity,3783.01


In [14]:
results_dir = joinpath(case_path, "results")
MacroEnergy.write_csv(joinpath(results_dir, "trad_cement_with_retrofit_capacity.csv"), capacity_results_df)

"/home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_retrofit/results/trad_cement_with_retrofit_capacity.csv"

In [15]:
## 8760 results
results_8760_df = DataFrame()
    # Get flows of each asset
for i in eachindex(system.assets)
    get_flows(system.assets[i], results_8760_df)
end

    # Function to get sorting key: returns the index of the first matching suffix or a large number
suffixes = ["cement", "co2"] # Define suffixes to reorder dataframe by
function suffix_sort_key(name)
    idx = findfirst(s -> endswith(name, s), suffixes)
    return isnothing(idx) ? length(suffixes) + 1 : idx  # Default to placing non-matching names at the end
end

    # Resort column order based on suffix
sorted_cols = sort(names(results_8760_df), by=suffix_sort_key) # Sort column names based on suffix priority
results_8760_df = results_8760_df[:, sorted_cols] # Reorder column names

#MacroEnergy.write_csv(joinpath(results_dir, "trad_cement_with_retrofit_8760_results.csv"), results_8760_df)

Row,trad_cement_MA_cement,trad_cement_CT_cement,trad_cement_ME_cement,echem_cement_MA_cement,echem_cement_CT_cement,echem_cement_ME_cement,trad_cement_MA_co2,trad_cement_CT_co2,trad_cement_ME_co2,ng_MA_co2,ng_CT_co2,ng_ME_co2,MA_to_CT,MA_to_ME,storage_MA_charge,storage_MA_discharge,storage_CT_charge,storage_CT_discharge,storage_ME_charge,storage_ME_discharge,ng_MA,ng_CT,ng_ME,solar_pv_MA,solar_pv_CT,onshore_wind_CT,onshore_wind_ME
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,38.9214,39.0204,38.636,11.9166,11.9488,11.2007,30.203,30.2798,29.9815,1462.16,765.648,30.7871,-1572.58,1305.77,-513.938,51.7531,-434.352,49.034,-417.648,38.2477,8075.37,4228.22,169.441,0.0,0.0,1.75927,2.65248
2,38.8682,38.9653,38.589,11.9529,11.983,11.2854,30.1618,30.237,29.945,1427.35,713.469,30.2976,-1378.95,1303.47,-555.815,51.2942,-461.505,48.7437,-468.48,36.5873,7883.11,3940.02,166.752,0.0,0.0,1.8366,2.30175
3,38.8382,38.9345,38.5602,11.9785,12.0108,11.3216,30.1384,30.2132,29.9227,1398.81,677.222,30.169,-1254.12,1305.73,-588.786,52.035,-477.742,49.1857,-513.711,37.1968,7725.48,3739.82,166.039,0.0,0.0,2.05251,2.45651
4,38.8288,38.9233,38.5508,11.9881,12.0161,11.3367,30.1311,30.2045,29.9154,1383.47,659.165,30.073,-1195.34,1305.85,-605.49,52.3687,-482.051,49.5148,-535.036,37.1372,7640.76,3640.1,165.504,0.0,0.0,1.99213,2.2824
5,38.8361,38.9256,38.5486,12.0049,12.0167,11.3119,30.1368,30.2063,29.9137,1381.92,656.363,30.1135,-1190.86,1312.06,-611.34,52.4943,-477.291,49.3902,-544.336,37.2077,7632.21,3624.63,165.736,0.0,0.0,1.34761,2.02097
6,38.8274,38.9172,38.5318,12.0273,12.0443,11.2959,30.1301,30.1997,29.9007,1392.68,668.932,29.9335,-1242.73,1313.44,-596.859,51.143,-458.333,48.2985,-527.492,36.1697,7691.65,3694.06,164.751,0.0,0.0,0.968561,1.77971
7,38.8597,38.9514,38.5669,11.9851,12.0063,11.2743,30.1551,30.2263,29.928,1413.56,697.542,29.9425,-1344.27,1310.95,-554.537,51.3726,-439.786,48.0274,-488.156,35.4117,7806.97,3852.07,164.797,0.0,0.0,1.15704,1.60935
8,38.8908,38.9853,38.5986,11.9509,11.9767,11.2414,30.1793,30.2526,29.9526,1432.25,726.585,30.1201,-1447.74,1312.26,-522.904,51.2979,-429.47,47.503,-457.941,35.8571,7910.19,4012.49,165.773,0.0,0.0,1.35001,1.59763
9,38.9052,39.0083,38.6032,11.9467,12.0079,11.1725,30.1905,30.2705,29.9561,1463.43,765.599,30.0873,-1605.78,1311.7,-450.769,50.0001,-372.497,47.8228,-405.257,37.9537,8082.41,4227.99,165.598,0.342702,0.420137,1.25151,1.40353


In [9]:
## System results
co2_node = MacroEnergy.get_nodes_sametype(system.locations, CO2)[1] # There is only 1 CO2 node
co2_captured_node = MacroEnergy.get_nodes_sametype(system.locations, CO2Captured)[1]
system_results_df = DataFrame(
    objective_value = MacroEnergy.objective_value(model),
    co2_emissions = MacroEnergy.value(sum(co2_captured_node.operation_expr[:exogenous])),
    #co2_captured = MacroEnergy.value(sum(co2_node.operation_expr[:emissions])),
)
mkpath(results_dir)
#MacroEnergy.write_csv(joinpath(results_dir,"trad_cement_with_retrofit_system_results.csv"), system_results_df)

"/home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_retrofit/results"

In [11]:
system_results_df

Row,objective_value,co2_emissions
,Float64,Float64
1,1.78832e9,0.0
